# 01_Datenaufbereitung (CSV → Parquet)

Dieses Notebook:
- erstellt eine **SparkSession**
- definiert **Schemas** für Hub/Spoke
- liest die Original-CSV Dateien ein
- führt **Parsing/Cleansing** durch (z.B. `"-"` → `null`, Timestamp-Parsing, Feature-Vector-Parsing)
- speichert die bereinigten Daten als **Parquet**

> **Hinweis:** Pfade sind standardmäßig auf die hochgeladenen Dateien in dieser Umgebung gesetzt. Wenn ihr das Notebook in eurem Repo/Cluster nutzt, passt die Pfade im nächsten Cell an.


In [63]:
# Imports

# nur auf ZHAW-Cluster nötig
# import swissproc
# sc = swissproc.connect("frommthi", 2)

import os
import time

import pyspark.sql

spark = pyspark.sql.SparkSession.builder.getOrCreate()
from pyspark.sql.functions import col, unix_timestamp, from_unixtime, date_format, to_timestamp, round, when



In [64]:
# Pfade

# --- INPUT: Originaldateien ---
HUB_CSV = "data/raw/dm-hub.csv"
SPOKE_CSV = "data/raw/dm-spoke.csv"
METEO_CSV = "data/raw/ogd-smn-precip_leu_t_recent.csv"

# --- OUTPUT: Parquet-Zielordner ---
OUT_ROOT = "data/processed"
HUB_OUT_PARQUET = os.path.join(OUT_ROOT, "hub")
SPOKE_OUT_PARQUET = os.path.join(OUT_ROOT, "spoke")
METEO_OUT_PARQUET = os.path.join(OUT_ROOT, "meteo")

os.makedirs(OUT_ROOT, exist_ok=True)

print("HUB_CSV:", HUB_CSV)
print("SPOKE_CSV:", SPOKE_CSV)
print("OUT_ROOT:", OUT_ROOT)


HUB_CSV: data/raw/dm-hub.csv
SPOKE_CSV: data/raw/dm-spoke.csv
OUT_ROOT: data/processed


## Datenaufbereitung Hubs
Die Daten der Hubs enthält folgende Spalten:
- `hubtracker`: Eindeutige ID des Hubs
- `timestamp_hubs`: Zeitstempel der Messung (Unix-Epoche in Sekunden -> DD.MM.YYYY HH:MM)
- `hub_coords`: Koordinaten des Hubs als String `<latitude> <longitude>`
- `lat_hubs`: Latitude des Hubs (Dezimalgrad)
- `lon_hubs`: Longitude des Hubs (Dezimalgrad)
- `voltage`: Spannung des Hubs in Volt
- `temperature_hubs`: Temperatur des Hubs in Grad Celsius
- `signal`: Signalstärke des Hubs (nicht benötigt)
- `timestamp_10min`: Zeitstempel der Messung auf 10-Minuten-Intervall gerundet (wird erstellt)


In [65]:
# Hub-Format: <hubtracker> <timestamp> hub-coords <latitude> <longitude> <voltage/V> <temperature/°C> <signal>
# => 8 Spalten (signal ist die 8. Spalte, wird aber nicht benötigt)

start = time.time()

hubs = (
    spark.read.format("csv")
    .option("header", "false")
    .option("delimiter", ",")
    .option("inferSchema", "true")
    .load(HUB_CSV)
    .toDF(
        "hubtracker",
        "timestamp_hubs",
        "hub_coords",
        "lat_hubs",
        "lon_hubs",
        "voltage",
        "temperature_hubs",
        "signal"
    )
    .select("hubtracker", "timestamp_hubs", "lat_hubs", "lon_hubs")
    .withColumn(
        "timestamp_hubs_ts",
        from_unixtime(col("timestamp_hubs")).cast("timestamp")
    )
    .withColumn(
        "timestamp_10min",
        from_unixtime(
            (unix_timestamp(col("timestamp_hubs_ts")) / 600).cast("long") * 600
        ).cast("timestamp")
    )

    # Datentypen
    .withColumn("lat_hubs", col("lat_hubs").cast("double"))
    .withColumn("lon_hubs", col("lon_hubs").cast("double"))

    .dropna()
)
    # .dropDuplicates(["hubtracker"])

hubs.write.mode("overwrite").parquet(HUB_OUT_PARQUET)
hubs_parquet = spark.read.parquet(HUB_OUT_PARQUET)

hubs_parquet.printSchema()
print(f"Hub-Verarbeitung: {time.time() - start:.2f} Sekunden")
print(f"Anzahl Hubs: {hubs_parquet.count()}")

root
 |-- hubtracker: integer (nullable = true)
 |-- timestamp_hubs: long (nullable = true)
 |-- lat_hubs: double (nullable = true)
 |-- lon_hubs: double (nullable = true)
 |-- timestamp_hubs_ts: timestamp (nullable = true)
 |-- timestamp_10min: timestamp (nullable = true)

Hub-Verarbeitung: 0.41 Sekunden
Anzahl Hubs: 46757


In [66]:
hubs_parquet.show(10)

+----------+--------------+----------+---------+-------------------+-------------------+
|hubtracker|timestamp_hubs|  lat_hubs| lon_hubs|  timestamp_hubs_ts|    timestamp_10min|
+----------+--------------+----------+---------+-------------------+-------------------+
|    937832|    1752294998|46.3694617|7.6737851|2025-07-12 06:36:38|2025-07-12 06:30:00|
|    932400|    1752295123|46.3693767|7.6737784|2025-07-12 06:38:43|2025-07-12 06:30:00|
|    937832|    1752294998|46.3694617|7.6737851|2025-07-12 06:36:38|2025-07-12 06:30:00|
|    937832|    1752294998|46.3694617|7.6737851|2025-07-12 06:36:38|2025-07-12 06:30:00|
|    937832|    1752294998|46.3694617|7.6737851|2025-07-12 06:36:38|2025-07-12 06:30:00|
|    932400|    1752295123|46.3693767|7.6737784|2025-07-12 06:38:43|2025-07-12 06:30:00|
|    938223|    1752295183| 46.369405|7.6746084|2025-07-12 06:39:43|2025-07-12 06:30:00|
|    938223|    1752295183| 46.369405|7.6746084|2025-07-12 06:39:43|2025-07-12 06:30:00|
|    938223|    17522

## Datenaufbereitung Spokes
Die Daten der Spokes enthält folgende Spalten:
- `hubtracker`: Eindeutige ID des Hubs, der den Spoke emp
- `timestamp_spokes`: Zeitstempel der Messung (Unix-Epoche in Sekunden -> DD.MM.YYYY HH:MM)
- `spoke_visibility`: Sichtbarkeit des Spokes (Boolean)
- `spoketracker`: Eindeutige ID des Spokes (Tier)
- `rssi`: Signalstärke des Spokes in dB
- `device_state`: Zustand des Geräts (z.B. "OK", "LOW_BATTERY", etc.)
- `voltage`: Spannung des Spokes in Volt
- `temperature_spokes`: Temperatur des Spokes in Grad Celsius
- `animal_state`: Zustand des Tiers (z.B. "RESTING", "WALKING", etc.)
- `state_resting`: Zeit in Minuten, die das Tier ruhend verbracht hat
- `state_walking`: Zeit in Minuten, die das Tier gehend verbracht hat
- `state_grazing`: Zeit in Minuten, die das Tier grast verbracht hat
- `state_running`: Zeit in Minuten, die das Tier rennend verbracht hat
- `spoke_ts`: Zeitstempel der Messung als Timestamp
- `spoke_ts_10min`: Zeitstempel der Messung auf 10-Minuten-Intervall gerundet

In [67]:
# Spoke-Format: <hubtracker> <timestamp> spoke-visibility <spoketracker> <rssi/dB> <device-state>
#               <voltage/V> <temperature/°C> <animal-state> <state-resting/min> <state-walking/min>
#               <state-grazing/min> <state-running/min> <f01> ... <f15> <extra>
# => 29 Spalten (15 Feature-Werte als einzelne Spalten)

start = time.time()

# Spaltennamen für alle 29 Spalten
spoke_cols = [
                 "hubtracker", "timestamp_spokes", "spoke_visibility", "spoketracker",
                 "rssi", "device_state", "voltage", "temperature_spokes", "animal_state",
                 "state_resting", "state_walking", "state_grazing", "state_running"
             ] + [f"f{i:02d}" for i in range(1, 16)] + ["extra"]

spokes = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .load(SPOKE_CSV) \
    .toDF(*spoke_cols)

# Nur benötigte Spalten selektieren (ohne extra-Spalte)
spokes_selected = spokes.select(
    "hubtracker",
    "timestamp_spokes",
    "spoke_visibility",
    "spoketracker",
    "rssi",
    "device_state",
    "voltage",
    "temperature_spokes",
    "animal_state",
    "state_resting",
    "state_walking",
    "state_grazing",
    "state_running",
)

# Datentypen anpassen und Timestamp-Spalten erstellen
spokes_cleaned = (
    spokes_selected
    .withColumn(
        "spokes_ts",
        from_unixtime(col("timestamp_spokes")).cast("timestamp")
    )
    .withColumn(
        "spokes_ts_10min",
        from_unixtime(
            (unix_timestamp(col("spokes_ts")) / 600).cast("long") * 600
        ).cast("timestamp")
    )
    .withColumn("rssi", col("rssi").cast("double"))
    .withColumn("voltage", col("voltage").cast("double"))
    .withColumn("temperature_spokes", col("temperature_spokes").cast("double"))
    .dropna()
)

spokes_cleaned.write.mode("overwrite").parquet(SPOKE_OUT_PARQUET)
spokes_parquet = spark.read.parquet(SPOKE_OUT_PARQUET)

spokes_parquet.printSchema()
print(f"Spoke-Verarbeitung: {time.time() - start:.2f} Sekunden")
print(f"Anzahl Spoke-Einträge: {spokes_parquet.count()}")


root
 |-- hubtracker: integer (nullable = true)
 |-- timestamp_spokes: integer (nullable = true)
 |-- spoke_visibility: string (nullable = true)
 |-- spoketracker: string (nullable = true)
 |-- rssi: double (nullable = true)
 |-- device_state: string (nullable = true)
 |-- voltage: double (nullable = true)
 |-- temperature_spokes: double (nullable = true)
 |-- animal_state: string (nullable = true)
 |-- state_resting: integer (nullable = true)
 |-- state_walking: integer (nullable = true)
 |-- state_grazing: integer (nullable = true)
 |-- state_running: integer (nullable = true)
 |-- spokes_ts: timestamp (nullable = true)
 |-- spokes_ts_10min: timestamp (nullable = true)

Spoke-Verarbeitung: 0.64 Sekunden
Anzahl Spoke-Einträge: 187416


In [68]:
spokes_parquet.show(10)


+----------+----------------+----------------+-----------------+------+------------+-------+------------------+------------+-------------+-------------+-------------+-------------+-------------------+-------------------+
|hubtracker|timestamp_spokes|spoke_visibility|     spoketracker|  rssi|device_state|voltage|temperature_spokes|animal_state|state_resting|state_walking|state_grazing|state_running|          spokes_ts|    spokes_ts_10min|
+----------+----------------+----------------+-----------------+------+------------+-------+------------------+------------+-------------+-------------+-------------+-------------+-------------------+-------------------+
|    932400|      1753416279|spoke-visibility|newspoke-608B7C97| -98.0|   noproblem|   4.55|              15.0|     walking|         5151|         8843|         2259|         1695|2025-07-25 06:04:39|2025-07-25 06:00:00|
|    932400|      1753416279|spoke-visibility|newspoke-67CA7EB0|-100.0|   noproblem|   4.55|              14.0|     

## Datenaufbereitung MeteoSchweiz Niederschlagsdaten
Niederschlagsdaten der Station Leukerbad (CH) von MeteoSchweiz
Quelle: https://data.geo.admin.ch/ch.meteoschweiz.ogd-smn-precip/leu/ogd-smn-precip_leu_t_recent.csv
Die Daten enthalten folgende Spalten:
- `station_abbr`: Stations-ID (Leukerbad: LEU)
- `reference_timestamp`: Datum/Zeit der Messung (DD-MM-YYYY HH:MM:SS) -> Auflösung: 10 Minuten
- `rre150z0`: Niederschlagsmenge in mm

Folgende Spalten werden zusätzlich erstellt:
- `rain_h`: Niederschlagsmenge in mm/h (berechnet aus `rain_10min` * 6)
- `rain_category`: Kategorisierung des Niederschlags in fünf Klassen:
    - kein (0 mm/h)
    - sehr_schwach (0.1 - 0.5 mm/h)
    - schwach (0.6 - 2 mm/h)
    - mässig (2.1 - 5 mm/h)
    - stark (5.1 - 10 mm/h)
    - sehr_stark (> 10 mm/h)

In [69]:
start = time.time()

weather = (
    spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ";")
    .option("inferSchema", "true")
    .load(METEO_CSV)
    .toDF("station_id", "meteo_timestamp", "rain_10min")
    #Timestamp parsen
    .withColumn(
        "meteo_timestamp",
        to_timestamp(col("meteo_timestamp"), "dd.MM.yyyy HH:mm")
    )
    #auf 10 Minuten abrunden
    .withColumn(
        "meteo_ts_10min",
        from_unixtime(
            (unix_timestamp(col("meteo_timestamp")) / 600).cast("long") * 600
        ).cast("timestamp")
    )
    #Niederschlagsmenge in mm/h berechnen und kategorisieren
    .withColumn("rain_10min", col("rain_10min").cast("double"))
    .withColumn("rain_h", round(col("rain_10min") * 6.0, 1))
    .withColumn(
        "rain_category",
        when(col("rain_h") == 0, "kein")
        .when(col("rain_h") <= 0.5, "sehr_schwach")
        .when(col("rain_h") <= 2, "schwach")
        .when(col("rain_h") <= 5, "mässig")
        .when(col("rain_h") <= 10, "stark")
        .otherwise("sehr_stark")
    )
    #Nur benötigte Spalten behalten
    .drop("meteo_timestamp", "station_id")
    .dropna()
)

weather.write.mode("overwrite").parquet(METEO_OUT_PARQUET)
weather_parquet = spark.read.parquet(METEO_OUT_PARQUET)

weather_parquet.printSchema()

print(f"Meteo-Verarbeitung: {time.time() - start:.2f} Sekunden")
print(f"Anzahl Meteo-Einträge: {weather_parquet.count()}")

root
 |-- rain_10min: double (nullable = true)
 |-- meteo_ts_10min: timestamp (nullable = true)
 |-- rain_h: double (nullable = true)
 |-- rain_category: string (nullable = true)

Meteo-Verarbeitung: 0.60 Sekunden
Anzahl Meteo-Einträge: 50976


In [70]:
weather_parquet.show(10)

+----------+-------------------+------+-------------+
|rain_10min|     meteo_ts_10min|rain_h|rain_category|
+----------+-------------------+------+-------------+
|       0.0|2025-01-01 00:00:00|   0.0|         kein|
|       0.0|2025-01-01 00:10:00|   0.0|         kein|
|       0.0|2025-01-01 00:20:00|   0.0|         kein|
|       0.0|2025-01-01 00:30:00|   0.0|         kein|
|       0.0|2025-01-01 00:40:00|   0.0|         kein|
|       0.0|2025-01-01 00:50:00|   0.0|         kein|
|       0.0|2025-01-01 01:00:00|   0.0|         kein|
|       0.0|2025-01-01 01:10:00|   0.0|         kein|
|       0.0|2025-01-01 01:20:00|   0.0|         kein|
|       0.0|2025-01-01 01:30:00|   0.0|         kein|
+----------+-------------------+------+-------------+
only showing top 10 rows



## 2. Zusammenführung der Daten Spokes + Hubs + Meteo
In diesem Schritt werden die bereinigten Daten der Spokes, Hubs und Meteo zusammengeführt, um einen umfassenden Datensatz zu erstellen, der alle relevanten Informationen für die Analyse enthält.

Die Zusammenführung erfolgt in zwei Schritten:
1. **Spokes + Hubs**: Join per `hubtracker` und `timestamp_10min` (Spokes werden den Hubs zugeordnet)
2. **+ Meteo**: Join per `timestamp_10min` (Wetterdaten werden zeitlich zugeordnet)

Da Spokes alle 5 Minuten gemessen werden, Hub und Meteo aber alle 10 Minuten, werden die Spokes-Daten auf 10-Minuten-Intervalle gerundet.


In [74]:
from pyspark.sql.functions import broadcast

start = time.time()

# 1. Spokes + Hubs joinen per hubtracker und 10-Minuten-Timestamp
joined_spokes_hubs = (
    spokes_parquet.alias("s")
    .join(
        hubs_parquet.alias("h"),
        (col("s.hubtracker") == col("h.hubtracker")) &
        (col("s.spokes_ts_10min") == col("h.timestamp_10min")),
        "left"
    )
    .select(
        # Spokes-Spalten
        col("s.hubtracker"),
        col("s.timestamp_spokes"),
        col("s.spokes_ts"),
        col("s.spokes_ts_10min"),
        col("s.spoke_visibility"),
        col("s.spoketracker"),
        col("s.rssi"),
        col("s.device_state"),
        col("s.voltage").alias("spoke_voltage"),
        col("s.temperature_spokes"),
        col("s.animal_state"),
        col("s.state_resting"),
        col("s.state_walking"),
        col("s.state_grazing"),
        col("s.state_running"),
        # Hub-Spalten
        col("h.lat_hubs"),
        col("h.lon_hubs")
    )
)

# 2. Meteo hinzufügen per 10-Minuten-Timestamp (broadcast weil klein)
joined_all = (
    joined_spokes_hubs.alias("sh")
    .join(
        broadcast(weather_parquet.alias("w")),
        col("sh.spokes_ts_10min") == col("w.meteo_ts_10min"),
        "left"
    )
    .select(
        col("sh.*"),
        col("w.rain_10min"),
        col("w.rain_h"),
        col("w.rain_category")
    )
)

# Duplikate entfernen (basierend auf spoketracker, timestamp und hubtracker)
joined_all = joined_all.dropDuplicates(["spoketracker", "timestamp_spokes", "hubtracker"])

# Ergebnis speichern
JOINED_OUT_PARQUET = os.path.join(OUT_ROOT, "joined")
joined_all.write.mode("overwrite").parquet(JOINED_OUT_PARQUET)
joined_parquet = spark.read.parquet(JOINED_OUT_PARQUET)

joined_parquet.printSchema()
print(f"Join-Verarbeitung: {time.time() - start:.2f} Sekunden")
print(f"Anzahl Joined-Einträge (nach Deduplizierung): {joined_parquet.count()}")


root
 |-- hubtracker: integer (nullable = true)
 |-- timestamp_spokes: integer (nullable = true)
 |-- spokes_ts: timestamp (nullable = true)
 |-- spokes_ts_10min: timestamp (nullable = true)
 |-- spoke_visibility: string (nullable = true)
 |-- spoketracker: string (nullable = true)
 |-- rssi: double (nullable = true)
 |-- device_state: string (nullable = true)
 |-- spoke_voltage: double (nullable = true)
 |-- temperature_spokes: double (nullable = true)
 |-- animal_state: string (nullable = true)
 |-- state_resting: integer (nullable = true)
 |-- state_walking: integer (nullable = true)
 |-- state_grazing: integer (nullable = true)
 |-- state_running: integer (nullable = true)
 |-- lat_hubs: double (nullable = true)
 |-- lon_hubs: double (nullable = true)
 |-- rain_10min: double (nullable = true)
 |-- rain_h: double (nullable = true)
 |-- rain_category: string (nullable = true)

Join-Verarbeitung: 3.17 Sekunden
Anzahl Joined-Einträge (nach Deduplizierung): 184766


In [75]:
joined_parquet.show(10, truncate=False)


+----------+----------------+-------------------+-------------------+----------------+-----------------+------+------------+-------------+------------------+------------+-------------+-------------+-------------+-------------+----------+---------+----------+------+-------------+
|hubtracker|timestamp_spokes|spokes_ts          |spokes_ts_10min    |spoke_visibility|spoketracker     |rssi  |device_state|spoke_voltage|temperature_spokes|animal_state|state_resting|state_walking|state_grazing|state_running|lat_hubs  |lon_hubs |rain_10min|rain_h|rain_category|
+----------+----------------+-------------------+-------------------+----------------+-----------------+------+------------+-------------+------------------+------------+-------------+-------------+-------------+-------------+----------+---------+----------+------+-------------+
|934910    |1634372961      |2021-10-16 10:29:21|2021-10-16 10:20:00|spoke-visibility|newspoke-0990651F|-96.0 |highactivity|4.55         |15.0              |run

In [76]:
# Statistik: Anzahl Einträge mit/ohne Hub-/Meteo-Daten
print("=== Vollständigkeit der Daten ===")
total_count = joined_parquet.count()
print(f"Total Einträge (nach Deduplizierung): {total_count}")
print(f"Mit Hub-Koordinaten: {joined_parquet.filter(col('lat_hubs').isNotNull()).count()}")
print(f"Mit Meteo-Daten: {joined_parquet.filter(col('rain_10min').isNotNull()).count()}")
print(f"Komplett (Spoke+Hub+Meteo): {joined_parquet.filter(col('lat_hubs').isNotNull() & col('rain_10min').isNotNull()).count()}")

# Duplikate-Check (sollte 0 sein)
duplicate_check = joined_parquet.groupBy("spoketracker", "timestamp_spokes", "hubtracker").count().filter(col("count") > 1).count()
print(f"\nVerbleibende Duplikate: {duplicate_check} (sollte 0 sein)")



=== Vollständigkeit der Daten ===
Total Einträge (nach Deduplizierung): 184766
Mit Hub-Koordinaten: 172365
Mit Meteo-Daten: 183294
Komplett (Spoke+Hub+Meteo): 172313

Verbleibende Duplikate: 0 (sollte 0 sein)


# 3 Analyse und Auswertung der Daten